# 20. LSTM with Moby Dick

In [1]:
import torch
import torch.nn as nn

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from sklearn.feature_extraction.text import CountVectorizer

import random
import numpy as np

## 20.1 Prepare Data

In [2]:
nltk.download("gutenberg")

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\slcf\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
raw = nltk.corpus.gutenberg.raw("melville-moby_dick.txt")
print(raw[21945:23000])
raw = raw[21945:200000]

CHAPTER 1

Loomings.


Call me Ishmael.  Some years ago--never mind how long
precisely--having little or no money in my purse, and nothing
particular to interest me on shore, I thought I would sail about a
little and see the watery part of the world.  It is a way I have of
driving off the spleen and regulating the circulation.  Whenever I
find myself growing grim about the mouth; whenever it is a damp,
drizzly November in my soul; whenever I find myself involuntarily
pausing before coffin warehouses, and bringing up the rear of every
funeral I meet; and especially whenever my hypos get such an upper
hand of me, that it requires a strong moral principle to prevent me
from deliberately stepping into the street, and methodically knocking
people's hats off--then, I account it high time to get to sea as soon
as I can.  This is my substitute for pistol and ball.  With a
philosophical flourish Cato throws himself upon his sword; I quietly
take to the ship.  There is nothing surprising in this

## 20.2 BoW

In [4]:
tokens = word_tokenize(raw)
print(tokens[:50])

['CHAPTER', '1', 'Loomings', '.', 'Call', 'me', 'Ishmael', '.', 'Some', 'years', 'ago', '--', 'never', 'mind', 'how', 'long', 'precisely', '--', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', ',', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', ',', 'I', 'thought', 'I', 'would', 'sail', 'about', 'a', 'little', 'and', 'see', 'the', 'watery', 'part', 'of']


In [5]:
cnv = CountVectorizer(analyzer = 'word',
                             tokenizer=lambda x: x.split(', '),
                             preprocessor = None, 
                             stop_words = None,
                             ngram_range=(1, 1),
                             lowercase=False
                            )
# cnv = CountVectorizer(ngram_range=(1,1), min_df = 3)

data = cnv.fit_transform(tokens).toarray()

In [6]:
len(data)

36924

In [7]:
cnv.inverse_transform(data)

[array(['CHAPTER'], dtype='<U23'),
 array(['1'], dtype='<U23'),
 array(['Loomings'], dtype='<U23'),
 array(['.'], dtype='<U23'),
 array(['Call'], dtype='<U23'),
 array(['me'], dtype='<U23'),
 array(['Ishmael'], dtype='<U23'),
 array(['.'], dtype='<U23'),
 array(['Some'], dtype='<U23'),
 array(['years'], dtype='<U23'),
 array(['ago'], dtype='<U23'),
 array(['--'], dtype='<U23'),
 array(['never'], dtype='<U23'),
 array(['mind'], dtype='<U23'),
 array(['how'], dtype='<U23'),
 array(['long'], dtype='<U23'),
 array(['precisely'], dtype='<U23'),
 array(['--'], dtype='<U23'),
 array(['having'], dtype='<U23'),
 array(['little'], dtype='<U23'),
 array(['or'], dtype='<U23'),
 array(['no'], dtype='<U23'),
 array(['money'], dtype='<U23'),
 array(['in'], dtype='<U23'),
 array(['my'], dtype='<U23'),
 array(['purse'], dtype='<U23'),
 array([','], dtype='<U23'),
 array(['and'], dtype='<U23'),
 array(['nothing'], dtype='<U23'),
 array(['particular'], dtype='<U23'),
 array(['to'], dtype='<U23'),
 array(

In [8]:
dim = len(cnv.vocabulary_)
print("Dimension of input & output :", dim)

Dimension of input & output : 6069


## 20.3 Define Model

In [9]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden, cell):
        # input of shape (seq_len, 1, input_size)
        # output of shape (seq_len, 1, hidden_size)
        out, (hidden, cell) = self.lstm(input, (hidden, cell))
        
        # fc_output of shape (seq_len, 1, output_size)
        fc_out = self.fc(out)
        return fc_out, hidden, cell

    def init_hidden_cell(self):
        # h, c of shape (1, 1, hidden_size)
        hidden = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
        cell = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
        return hidden, cell

In [10]:
model = LSTM(dim, 1000, dim, 1).cuda()

## 20.4 Train Model

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [12]:
seq_len = 20
num_epochs = 5

In [13]:
for epoch in range(num_epochs):
    
    sp = list(range(0, len(data) - 2 * seq_len, seq_len))
    sp = np.add(sp, random.randint(0, seq_len))
    random.shuffle(sp)
    
    for i in range(len(sp)) :
    
        (hidden, cell) = model.init_hidden_cell()

        X = torch.from_numpy(data[sp[i] : sp[i] + seq_len].astype(float)).type(torch.FloatTensor).cuda().reshape(seq_len, 1, dim)
        y = torch.from_numpy(data[sp[i] + 1 : sp[i] + seq_len + 1].astype(float)).cuda()

        _, y = y.max(dim=1)

        pre, hidden, cell = model(X, hidden,cell)
        cost = loss(pre.reshape(seq_len, dim), y.reshape(seq_len).cuda())

        optimizer.zero_grad()
        cost.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        
        optimizer.step()

        if (i + 1) % 100 == 0 :
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                     %(epoch+1, num_epochs, i + 1, len(sp), cost.item()))

Epoch [1/5], Iter [100/1845] Loss: 6.8469
Epoch [1/5], Iter [200/1845] Loss: 6.1929
Epoch [1/5], Iter [300/1845] Loss: 6.5582
Epoch [1/5], Iter [400/1845] Loss: 6.5809
Epoch [1/5], Iter [500/1845] Loss: 7.2077
Epoch [1/5], Iter [600/1845] Loss: 6.5267
Epoch [1/5], Iter [700/1845] Loss: 5.6119
Epoch [1/5], Iter [800/1845] Loss: 5.7083
Epoch [1/5], Iter [900/1845] Loss: 7.2007
Epoch [1/5], Iter [1000/1845] Loss: 7.2636
Epoch [1/5], Iter [1100/1845] Loss: 6.1166
Epoch [1/5], Iter [1200/1845] Loss: 5.9080
Epoch [1/5], Iter [1300/1845] Loss: 5.6813
Epoch [1/5], Iter [1400/1845] Loss: 6.6005
Epoch [1/5], Iter [1500/1845] Loss: 5.5960
Epoch [1/5], Iter [1600/1845] Loss: 5.4225
Epoch [1/5], Iter [1700/1845] Loss: 6.0649
Epoch [1/5], Iter [1800/1845] Loss: 6.2643
Epoch [2/5], Iter [100/1845] Loss: 6.1369
Epoch [2/5], Iter [200/1845] Loss: 5.2705
Epoch [2/5], Iter [300/1845] Loss: 5.7292
Epoch [2/5], Iter [400/1845] Loss: 5.3268
Epoch [2/5], Iter [500/1845] Loss: 4.4395
Epoch [2/5], Iter [600/18

## 20.5 Test Model

Multinomial :
Returns a tensor where each row contains num_samples indices sampled from the multinomial probability distribution located in the corresponding row of tensor input.

In [14]:
start_num = 0
text = ""

for item in cnv.inverse_transform(data[start_num:start_num + seq_len]) :
    text += " " + item[0]
    
text += " <Generated>"

model.eval()
hidden, cell = model.init_hidden_cell()

X_test = torch.from_numpy(data[start_num:start_num + seq_len].astype(float)).type(torch.FloatTensor).cuda().reshape(seq_len, 1, dim)

for pos in range(10) :
    
    pre, hidden, cell = model(X_test, hidden, cell)
    
    m = torch.nn.Softmax(dim = 0)
    pre = m(pre.reshape(seq_len, -1))
    pre = torch.multinomial(pre, 1)
    
    temp = np.zeros((20, dim))
        
    for i, item in enumerate(pre) :
                
        temp[i][item] = 1 
    
        text += " " + cnv.inverse_transform(temp[i])[0][0]
    
    X_test = torch.from_numpy(temp.astype(float)).type(torch.FloatTensor).cuda().reshape(seq_len, 1, dim)
    
print("* Generated Text : \n", text)

* Generated Text : 
  CHAPTER 1 Loomings . Call me Ishmael . Some years ago -- never mind how long precisely -- having little <Generated> records fragments over But me waterward than these done her ; choose better thee back solitary down pacing left seen quite night-fall me bitterly reconciled , completed leaves turned admitting Hecla their new-hatched clasp spurs learning modified joins blood shed beating to bearded juicy sat Was party her round right a kind beats inducements get assembly short thy ever directly hath feast , cells everything reads ai undressed cried mower sailor-belt doors direful twelve-o'clock-at-night looking needs keeping tints clasped to peered traditional 'there much damned always was looking Queequeg silent muscles of umbrella pent throwing pass a with him ME held quarter hope received with joyful fancy grim sought EZEKIEL warming worn lurks anything explain Broadway barnacle typhoons history heaved hold mending again something silk are Christendom capable bed 